In [1]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook
import engarde.decorators as ed

from linearmodels import PanelOLS

from library import test_data
@ed.verify(test_data.allyearsandtpsd)
@ed.verify(test_data.alldois)
def load(data):
    return data

from library import print_statistics

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/panel/data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


In [2]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)

In [3]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'gdid.csv'),
                  sep=",", low_memory= False)
#load(data)
data.tail()
print(data[data.doi == True].district.nunique())
data = data[data.doi == True]
data.sample()

822


,Unnamed: 0,year,campus,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,...,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,treatpost
13984,13984,2013,126901001,ALVARADO H S,N,126901,ALVARADO ISD,N,M,NaN,...,0,1,0,0,1,NaN,NaN,NaN,NaN,False


In [4]:
data['yearpost'] = data.year - data.doi_year
data.yearpost.value_counts()

 0.0    6346
-1.0    6329
-2.0    6293
-3.0    6275
-4.0    6263
 1.0    6129
-5.0    5356
 2.0    5190
-6.0    1152
 3.0     938
-7.0     201
Name: yearpost, dtype: int64

In [5]:
#convert year to datetime
df = data.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
#add column year to index
df = data.set_index(['year', 'campus'])
#swap indexes
df.index = df.index.swaplevel(0,1)
df[['district', 'doi_year','treatpost',]].tail(5)

,,district,doi_year,treatpost
campus,year,,,
253901105,2019,253901,2018.0,True
253901106,2019,253901,2018.0,True
253901107,2019,253901,2018.0,True
254902001,2019,254902,2018.0,True
254902101,2019,254902,2018.0,True


In [6]:
mod = PanelOLS.from_formula('sec_reading ~ + 1 + treatpost + students_hisp + students_num + TimeEffects + EntityEffects', df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            sec_reading   R-squared:                        0.0101
Estimator:                   PanelOLS   R-squared (Between):              0.1714
No. Observations:               23605   R-squared (Within):               0.0063
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.1540
Time:                        13:54:06   Log-likelihood                   -5404.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      69.176
Entities:                        3349   P-value                           0.0000
Avg Obs:                       7.0484   Distribution:                 F(3,20246)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             17.179
                            

# Heterogenous effects

In [7]:
### Generate variable for number of hispanic students in year before the district declares

In [8]:
data.teachers_turnover_ratio_d

3         8.1740
4        10.8636
5        10.8636
6        10.8636
7        10.8636
          ...   
69234     9.8000
69235     9.8000
69236     9.8000
69242    30.4000
69243    30.4000
Name: teachers_turnover_ratio_d, Length: 50951, dtype: float64

In [9]:
data_pre = data.loc[data.yearpost == -1]
data_pre = data_pre.rename(columns = {'students_hisp': 'students_hisp_pre',
                                     'teachers_turnover_ratio_d': 'teacher_turnover_pre'})
data_pre['low_avescores_pre'] = np.where(data_pre.avescores < data_pre.avescores.quantile(.25),1, 0)
data_pre['high_avescores_pre'] = np.where(data_pre.avescores > data_pre.avescores.quantile(.75),1,0)
data_pre = data_pre[['campus','students_hisp_pre', 'teacher_turnover_pre', 'low_avescores_pre', 'high_avescores_pre']]
data_hte = data.merge(data_pre, on = 'campus', how = 'left')
data_hte['majority_hisp_pre'] = np.where(data_hte.students_hisp_pre > .6, 1, 0)
data_hte['high_turnover'] = np.where(data_hte.teacher_turnover_pre > 20, 1, 0)
data_hte

,Unnamed: 0,year,campus,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,...,teachers_msdegree,teachers_phddegree,treatpost,yearpost,students_hisp_pre,teacher_turnover_pre,low_avescores_pre,high_avescores_pre,majority_hisp_pre,high_turnover
0,3,2012,109901001,ABBOTT SCHOOL,N,109901,ABBOTT ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.127208,16.5,0.0,1.0,0,0
1,4,2012,95901003,ABERNATHY DAEP,N,95901,ABERNATHY ISD,N,NaN,NaN,...,NaN,NaN,False,-5.0,1.000000,13.5,0.0,0.0,1,0
2,5,2012,95901101,ABERNATHY EL,N,95901,ABERNATHY ISD,N,NaN,NaN,...,NaN,NaN,False,-5.0,0.620596,13.5,0.0,0.0,1,0
3,6,2012,95901001,ABERNATHY H S,N,95901,ABERNATHY ISD,N,NaN,NaN,...,NaN,NaN,False,-5.0,0.495370,13.5,0.0,0.0,0,0
4,7,2012,95901041,ABERNATHY J H,N,95901,ABERNATHY ISD,N,NaN,NaN,...,NaN,NaN,False,-5.0,0.567708,13.5,0.0,1.0,0,0
5,8,2012,221901001,ABILENE H S,N,221901,ABILENE ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.458668,13.1,0.0,0.0,0,0
6,9,2012,221901010,ACADEMY FOR TECHNOLOGY ENGINEERING,N,221901,ABILENE ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.379501,13.1,0.0,0.0,0,0
7,10,2012,221901142,ACADIA ABILENE BEHAVIORAL HEALTH,N,221901,ABILENE ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.428571,13.1,0.0,0.0,0,0
8,11,2012,221901102,AUSTIN EL,N,221901,ABILENE ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.336134,13.1,0.0,1.0,0,0
9,12,2012,221901153,BASSETTI EL,N,221901,ABILENE ISD,N,NaN,NaN,...,NaN,NaN,False,-6.0,0.372624,13.1,0.0,0.0,0,0


In [10]:
data_pre = data.loc[data.year == 2016]
data_pre['rural_pre'] = np.where(data_pre.type_description == "RURAL", 1, 0)
data_pre['urban_pre'] = np.where(data_pre.type_description == "URBAN", 1, 0)
data_pre = data_pre[['campus','rural_pre', 'urban_pre']]
data_hte = data_hte.merge(data_pre, on = 'campus', how = 'left')
#df_hte[['students_hisp', 'students_hisp_pre', 'yearpost']].sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [11]:
#convert year to datetime
df_hte = data_hte.reset_index()
df_hte['year'] = pd.to_datetime(df_hte['year'], format='%Y')
#add column year to index
df_hte = data_hte.set_index(['year', 'campus'])
#swap indexes
df_hte.index = df_hte.index.swaplevel(0,1)
df_hte[['district', 'doi_year','treatpost',]].tail(20)

,,district,doi_year,treatpost
campus,year,,,
251901101,2019,251901,2017.0,True
251901104,2019,251901,2017.0,True
252901001,2019,252901,2017.0,True
252901002,2019,252901,2017.0,True
252901041,2019,252901,2017.0,True
252901101,2019,252901,2017.0,True
252901104,2019,252901,2017.0,True
252901105,2019,252901,2017.0,True
252902002,2019,252902,2017.0,True


# Majority Hispanic

In [12]:
df_hte['treatpost_hisp'] = df_hte.treatpost * df_hte.majority_hisp_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_hisp + students_hisp + students_num + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0097
Estimator:                   PanelOLS   R-squared (Between):              0.0984
No. Observations:               45633   R-squared (Within):              -0.0307
Date:                Mon, Oct 28 2019   R-squared (Overall):              0.0849
Time:                        13:54:23   Log-likelihood                   -4213.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      96.672
Entities:                        6088   P-value                           0.0000
Avg Obs:                       7.4956   Distribution:                 F(4,39534)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             30.607
                            

In [28]:
df_hte['treatpost_hisp'] = df_hte.treatpost * df_hte.majority_hisp_pre
df_hte['treatpost_hisp_rural'] = df_hte.treatpost_hisp * df_hte.rural_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_hisp + treatpost_hisp_rural + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0054
Estimator:                   PanelOLS   R-squared (Between):             -0.0102
No. Observations:               45033   R-squared (Within):              -0.0242
Date:                Mon, Oct 28 2019   R-squared (Overall):             -0.0119
Time:                        14:00:33   Log-likelihood                   -4403.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      70.884
Entities:                        5844   P-value                           0.0000
Avg Obs:                       7.7059   Distribution:                 F(3,39179)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             25.803
                            

In [27]:
df_hte[df_hte.majority_hisp_pre == 1][['district', 'type_description', 'majority_hisp_pre']]

,,district,type_description,majority_hisp_pre
campus,year,,,
95901003,2012,95901,RURAL,1
95901101,2012,95901,RURAL,1
221901006,2012,221901,OTHER CENTRAL CITY,1
221901152,2012,221901,OTHER CENTRAL CITY,1
221901125,2012,221901,OTHER CENTRAL CITY,1
180903001,2012,180903,RURAL,1
178901101,2012,178901,RURAL,1
178901001,2012,178901,RURAL,1
101902001,2012,101902,MAJOR SUBURBAN,1


# Rural

In [13]:
df_hte['treatpost_rural'] = df_hte.treatpost * df_hte.rural_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_rural + rural_pre +  students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          Intercept, rural_pre

Set drop_absorbed=True to automatically drop absorbed variables.


# Performance

In [76]:
df_hte['treatpost_low'] = df_hte.treatpost * df_hte.low_avescores_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_low + students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0055
Estimator:                   PanelOLS   R-squared (Between):              0.1075
No. Observations:               44625   R-squared (Within):              -0.0371
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0922
Time:                        10:59:37   Log-likelihood                   -4385.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      53.286
Entities:                        5795   P-value                           0.0000
Avg Obs:                       7.7006   Distribution:                 F(4,38819)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             14.691
                            

In [77]:
df_hte['treatpost_high'] = df_hte.treatpost * df_hte.high_avescores_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_high+ students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0045
Estimator:                   PanelOLS   R-squared (Between):              0.1131
No. Observations:               44625   R-squared (Within):              -0.0372
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0979
Time:                        10:59:39   Log-likelihood                   -4406.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      44.207
Entities:                        5795   P-value                           0.0000
Avg Obs:                       7.7006   Distribution:                 F(4,38819)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             12.016
                            

# Teacher Turnover

In [78]:
df_hte['treatpost_turnover'] = df_hte.treatpost * df_hte.high_turnover
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_turnover + students_hisp + students_num + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0049
Estimator:                   PanelOLS   R-squared (Between):              0.1116
No. Observations:               45633   R-squared (Within):              -0.0331
Date:                Tue, Oct 15 2019   R-squared (Overall):              0.0985
Time:                        11:00:01   Log-likelihood                   -4324.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      48.184
Entities:                        6088   P-value                           0.0000
Avg Obs:                       7.4956   Distribution:                 F(4,39534)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             13.505
                            